In [1]:
from birb_dataset import get_bird_dataset
from denoising_diffusion_pytorch import GaussianDiffusion, Unet, Trainer

/home/bigbrainman/deepvenv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ds = get_bird_dataset('./data/birds', 64)

In [3]:
ds[0]

IVORY GULL
torch.Size([3, 64, 64])
torch.Size([3, 64, 64])


tensor([[[-0.7942,  1.8868, -0.0878,  ...,  0.2824,  0.7125, -2.7449],
         [ 0.2788, -0.1314, -1.0418,  ..., -0.0585, -0.8161,  0.0166],
         [-2.1795, -0.5832,  0.5633,  ..., -0.8500,  0.2357, -0.0633],
         ...,
         [-1.3578,  1.3880, -1.2938,  ..., -1.1440,  0.3665, -0.1618],
         [-0.8194, -1.0541, -0.6816,  ...,  0.1713,  0.2825, -1.6807],
         [-0.3621,  0.1195,  0.1958,  ..., -1.6605, -0.4418,  0.4506]],

        [[-0.7942,  1.8868, -0.0878,  ...,  0.2824,  0.7125, -2.7449],
         [ 0.2788, -0.1314, -1.0418,  ..., -0.0585, -0.8161,  0.0166],
         [-2.1795, -0.5832,  0.5633,  ..., -0.8500,  0.2357, -0.0633],
         ...,
         [-1.3578,  1.3880, -1.2938,  ..., -1.1440,  0.3665, -0.1618],
         [-0.8194, -1.0541, -0.6816,  ...,  0.1713,  0.2825, -1.6807],
         [-0.3621,  0.1195,  0.1958,  ..., -1.6605, -0.4418,  0.4506]],

        [[-0.7942,  1.8868, -0.0878,  ...,  0.2824,  0.7125, -2.7449],
         [ 0.2788, -0.1314, -1.0418,  ..., -0